<a href="https://colab.research.google.com/github/k00lk0der1/synthpoppp/blob/main/Pipeline_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Change local time to IST from Colab default of GMT

In [ ]:
# Check your currnet timezone directory

!ls -al /usr/share/zoneinfo/Asia/Kolkata

lrwxrwxrwx 1 root root 8 Nov 30 19:08 /usr/share/zoneinfo/Asia/Kolkata -> Calcutta


In [ ]:
# Update the localtime on Google Colab 
# Source: https://stackoverflow.com/questions/57398787/different-timezone-in-google-colab

!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!date

Sat Jan 14 18:52:33 IST 2023


In [ ]:
# from datetime import datetime

# # datetime object containing current date and time
# now = datetime.now()
 
# print("now =", now)

now = 2023-01-14 13:19:19.832398


#Download dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/k00lk0der1/bsim_synthpop.git
!pip3 install -r /content/bsim_synthpop/synthpop/requirements-dev.txt
!pip3 install scipy==1.4.1
!pip3 install geopandas

In [ ]:
!wget https://data.worldpop.org/GIS/Population_Density/Global_2000_2020_1km/2020/IND/ind_pd_2020_1km_ASCII_XYZ.zip
!unzip ind_pd_2020_1km_ASCII_XYZ.zip

In [ ]:
from bsim_synthpop.base_pop import IPU
from bsim_synthpop.arbitrary import POSSIBLE_JOB_LABELS
from bsim_synthpop.geographical_distributions import HLatHlongAgeAddition, JobsPlacesAddition, PopulationDensitySampler
import pandas as pd
import numpy as np

#

In [ ]:
###Configuration

region_name = "haryana/ambala"
state_id = 6 ####### select state number and name appropriately
district_ids = [] #select district number

district_wise_filtering = False

population_density_filename = "ind_pd_2020_1km_ASCII_XYZ.csv"

# working_dir = "/content/drive/MyDrive/data/synthetic/"

working_dir = "/content/drive/MyDrive/PhD_Data/Synth_Pop/Analysis/Source_Files/District_wise_source_files/"

ihds_individuals_filename = working_dir+"36151-0001-Data.tsv"
ihds_household_filename = working_dir+"36151-0002-Data.tsv"

householdh_marginal_filename = working_dir+region_name+"/household_marg.csv"
individuals_marginal_filename = working_dir+region_name+"/person_marg.csv"

admin_units_geojson_filename = working_dir+region_name+"/admin_units.geojson"
admin_units_population_filename = working_dir+region_name+"/admin_unit_wise_pop.csv"

if(district_wise_filtering):
    synthetic_population_output_filename = working_dir+region_name+"/synthetic.csv"
else:
    synthetic_population_output_filename = working_dir+region_name+"/synthetic_state_wise.csv"

In [ ]:
ihds_individuals_data = pd.read_csv(ihds_individuals_filename, sep='\t')

filtered_ihds_individuals_data = ihds_individuals_data.loc[ihds_individuals_data.STATEID==state_id]

ihds_households_data = pd.read_csv(ihds_household_filename, sep='\t')

filtered_ihds_households_data = ihds_households_data.loc[ihds_households_data.STATEID==state_id] 

if(district_wise_filtering):
    filtered_ihds_individuals_data = filtered_ihds_individuals_data[filtered_ihds_individuals_data['DISTID'].apply(lambda x : x in district_ids)]   
    filtered_ihds_households_data = filtered_ihds_households_data[filtered_ihds_households_data['DISTID'].apply(lambda x : x in district_ids)]

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,6,9,10,11,12,13,14,15,16,17,18,22,27,29,33,34,39,44,48,49,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,74,75,76,78,79,80,88,91,105,150,151,152,154,158,159,166,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,194,196,197,198,199,200,201,202,203,204,205,217,218,219,220,221,222,223,224,225,226,227,228,229,230,234,235,236,237,240,241,243,245,247,248,249,250,275,279,283,287,288,289,290,291,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,320,322,334) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14,15,16,17,18,19,20,21,24,25,26,28,31,35,36,113,114,115,116,117,118,119,120,121,122,126,186,187,188,189,190,191,192,193,194,212,213,214

In [ ]:
filtered_ihds_households_data

,SURVEY,STATEID,DISTID,PSUID,HHID,HHSPLITID,IDPSU,IDHH,WT,INDWT,...,NWKANY5,INCNONAG,INCAGLAB,INCSALARY,INCNREGA,INCNONNREGA,NNR,HHEDUC,HHEDUCM,HHEDUCF
4451,2,6,1,1,21,1,60101,601010211,7223.443234,43340.659403,...,1,22500,0,0,0,22500,0,0,0,0
4452,2,6,1,1,31,1,60101,601010311,7223.443234,28893.772935,...,2,0,0,0,0,0,0,0,0,0
4453,2,6,1,1,32,1,60101,601010321,7223.443234,28893.772935,...,2,0,0,0,0,0,0,8,8,5
4454,2,6,1,1,41,1,60101,601010411,7223.443234,86681.318805,...,4,0,0,48000,0,0,0,12,12,12
4455,2,6,1,1,51,1,60101,601010511,7223.443234,57787.545870,...,1,0,0,120000,0,0,0,6,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6252,2,6,19,9,110,1,61909,619091101,4407.666667,8815.333333,...,1,0,0,72000,0,0,0,10,9,10
6253,2,6,19,9,110,2,61909,619091102,4407.666667,13223.000000,...,1,36000,0,0,0,36000,0,12,12,12
6254,2,6,19,9,130,1,61909,619091301,4407.666667,22038.333333,...,3,0,0,138000,0,0,0,15,15,8
6255,2,6,19,9,140,1,61909,619091401,4407.666667,35261.333333,...,3,0,0,304800,0,0,0,16,15,16


In [ ]:
filtered_ihds_individuals_data

,SURVEY,STATEID,DISTID,PSUID,HHID,HHSPLITID,PERSONID,IDPSU,IDHH,IDPERSON,...,MG8,MG9NM,MG10,MG11,MGYEAR5,NMIG5,MGMONTHS5,MGYEAR1,NMIG1,MGMONTHS1
21882,2,6,1,1,21,1,1,60101,601010211,60101021101,...,,,,,0,,,0,,
21883,2,6,1,1,21,1,2,60101,601010211,60101021102,...,,,,,0,,,0,,
21884,2,6,1,1,21,1,3,60101,601010211,60101021103,...,,,,,0,,,0,,
21885,2,6,1,1,21,1,4,60101,601010211,60101021104,...,,,,,0,,,0,,
21886,2,6,1,1,21,1,5,60101,601010211,60101021105,...,,,,,0,,,0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31445,2,6,19,9,140,1,8,61909,619091401,61909140108,...,,,,,0,,,0,,
31446,2,6,19,9,150,1,1,61909,619091501,61909150101,...,,,,,0,,,0,,
31447,2,6,19,9,150,1,2,61909,619091501,61909150102,...,,,,,0,,,0,,
31448,2,6,19,9,150,1,3,61909,619091501,61909150103,...,,,,,0,,,0,,


In [ ]:
ipu_object = IPU()
synthetic_households, synthetic_individuals, synthetic_stats = ipu_object.generate_data(filtered_ihds_individuals_data, filtered_ihds_households_data, householdh_marginal_filename, individuals_marginal_filename)

display(synthetic_individuals)
display(synthetic_households)

hlat_hlong_age_object = HLatHlongAgeAddition(admin_units_geojson_filename, admin_units_population_filename, population_density_filename)
new_synthetic_population = hlat_hlong_age_object.perform_transforms(synthetic_individuals, synthetic_households)

display(new_synthetic_population)

jobs_places_object = JobsPlacesAddition(POSSIBLE_JOB_LABELS, admin_units_geojson_filename, 10000, 10000, population_density_filename, 'zipf', 'default', 'default')
new_synthetic_population = jobs_places_object.perform_transforms(new_synthetic_population) 

new_synthetic_population.to_csv(synthetic_population_output_filename, index=False)

,district,serialno,mem_id,SexLabel,Age,religion,caste
21882,1,601010211,1,Male,35to39,Hindu,Other
21883,1,601010211,2,Female,30to34,Hindu,Other
21884,1,601010211,3,Female,10to14,Hindu,Other
21885,1,601010211,4,Female,5to9,Hindu,Other
21886,1,601010211,5,Male,5to9,Hindu,Other
...,...,...,...,...,...,...,...
31445,19,619091401,8,Female,0to4,Hindu,Other
31446,19,619091501,1,Male,70to74,Hindu,Other
31447,19,619091501,2,Female,60to64,Hindu,Other
31448,19,619091501,3,Male,25to29,Hindu,Other


,district,serialno,hhsize
4451,1,601010211,hhsize_6
4452,1,601010311,hhsize_4
4453,1,601010321,hhsize_4
4454,1,601010411,hhsize_1114
4455,1,601010511,hhsize_710
...,...,...,...
6252,19,619091101,hhsize_2
6253,19,619091102,hhsize_3
6254,19,619091301,hhsize_5
6255,19,619091401,hhsize_710


district             distid   hhsize                                      \
         Unnamed: 1_level_1 hhsize_1 hhsize_2 hhsize_3 hhsize_4 hhsize_5   
ambala                   70   6176.0  14452.0  25456.0  55634.0  48970.0   

district                                             
         hhsize_6 hhsize_710 hhsize_1114 hhsize_15p  
ambala    31558.0    34675.0      4508.0     1085.0

district distid  total_pop  SexLabel                Age                     \
         distid  total_pop      Male    Female     0to4     5to9    10to14   
ambala       70  1128350.0  598703.0  529647.0  89510.0  94872.0  100432.0   

district                                ...  religion                    \
            15to19    20to24    25to29  ...     Hindu  Muslim Christian   
ambala    110758.0  116232.0  106055.0  ...  137047.0  7693.0    1094.0   

district                                   caste             
             Sikh Buddhist   Jain  Other      SC ST   Other  
ambala    21888.0     50.0  139.0  162.0  296246  0  832104  

[1 rows x 31 columns]

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/content/bsim_synthpop/synthpop/synthpop/ipu/ipu.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  adj = constraint / float((column * weights).sum())


...

100%|██████████| 2125870/2125870 [01:12<00:00, 29150.09it/s]


,district,serialno,mem_id,SexLabel,Age,religion,caste,sample_geog,cat_id,geog,household_id,H_Lat,H_Lon,AdminUnitName,AdminUnitLatitude,AdminUnitLongitude
0,1,601010211,1,Male,37,Hindu,Other,1,158,ambala,152568,30.413445,77.037623,ambala,30.375201,76.782122
1,1,601010211,2,Female,34,Hindu,Other,1,494,ambala,152568,30.413445,77.037623,ambala,30.375201,76.782122
2,1,601010211,3,Female,14,Hindu,Other,1,410,ambala,152568,30.413445,77.037623,ambala,30.375201,76.782122
3,1,601010211,4,Female,6,Hindu,Other,1,389,ambala,152568,30.413445,77.037623,ambala,30.375201,76.782122
4,1,601010211,5,Male,9,Hindu,Other,1,32,ambala,152568,30.413445,77.037623,ambala,30.375201,76.782122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065038,18,618090501,1,Male,42,Jain,Other,1,194,ambala,148657,30.529442,76.998173,ambala,30.375201,76.782122
1065039,18,618090501,4,Male,14,Jain,Other,1,68,ambala,148657,30.529442,76.998173,ambala,30.375201,76.782122
1065040,18,618090501,2,Female,42,Jain,Other,1,551,ambala,148657,30.529442,76.998173,ambala,30.375201,76.782122
1065041,18,618090501,3,Female,19,Jain,Other,1,446,ambala,148657,30.529442,76.998173,ambala,30.375201,76.782122


100%|██████████| 4010402/4010402 [01:31<00:00, 43777.69it/s]


10001


100%|██████████| 100000/100000 [00:03<00:00, 28711.78it/s]
/content/bsim_synthpop/geographical_distributions.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['JobType'] = np.random.choice(self.job_type_list, size=(adults.shape[0],))
/content/bsim_synthpop/geographical_distributions.py:200: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adults['WorksAtSameCategory'] = np.random.uniform(size=(adults.shape[0],))>0.05
100%|██████████| 186/186 [00:25<00:00,  7.19it/s]
/content/bsim_synthpop/geographic

In [ ]:
df = new_synthetic_population
pop_size=len(df)

,district,serialno,mem_id,SexLabel,Age,religion,caste,sample_geog,cat_id,geog,...,WorksAtSameCategory,WorkplaceID,W_Lat,W_Lon,SchoolID,School_Lat,School_Lon,PublicPlaceID,PublicPlaceLat,PublicPlaceLong
0,1,601010211,1,Male,37,Hindu,Other,1,158,ambala,...,True,2001000000068,30.35835,76.963553,NaN,NaN,NaN,3001000001614,30.417883,77.067544
1,6,606040511,2,Female,60,Hindu,Other,1,620,ambala,...,True,2001000005784,30.151124,77.044531,NaN,NaN,NaN,3001000001614,30.417883,77.067544
2,8,608030811,2,Female,59,Muslim,Other,1,602,ambala,...,True,2001000002634,30.336813,76.764069,NaN,NaN,NaN,3001000001614,30.417883,77.067544
3,13,613050401,4,Female,21,Hindu,Other,1,452,ambala,...,True,2001000008742,30.204226,77.025145,NaN,NaN,NaN,3001000001614,30.417883,77.067544
4,12,612041001,1,Male,49,Hindu,Other,1,200,ambala,...,True,2001000001679,30.210001,76.98632,NaN,NaN,NaN,3001000001614,30.417883,77.067544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065038,5,605031911,5,Male,7,Sikh,Other,1,41,ambala,...,True,NaN,NaN,NaN,2.001000e+12,30.314988,76.968826,3001000005288,30.312996,77.004969
1065039,5,605041011,9,Male,6,Sikh,Other,1,41,ambala,...,True,NaN,NaN,NaN,2.001000e+12,30.314988,76.968826,3001000005288,30.312996,77.004969
1065040,9,609100301,5,Male,18,Hindu,Other,1,74,ambala,...,True,NaN,NaN,NaN,2.001000e+12,30.311981,77.024669,3001000005288,30.312996,77.004969
1065041,2,602080209,4,Female,4,Hindu,SC,1,366,ambala,...,True,NaN,NaN,NaN,2.001000e+12,30.390674,76.835168,3001000008889,30.423889,76.827809


In [ ]:
ls -ltr /content/drive/MyDrive/PhD_Data/Synth_Pop/Analysis/Source_Files/District_wise_source_files/haryana/ambala

total 248094
-rw------- 1 root root        66 Jan 14 09:00 admin_unit_wise_pop.csv
-rw------- 1 root root      7816 Jan 14 09:00 admin_units.geojson
-rw------- 1 root root       262 Jan 14 09:01 household_marg.csv
-rw------- 1 root root       636 Jan 14 11:09 person_marg.csv
-rw------- 1 root root 254037441 Jan 14 13:15 synthetic_state_wise.csv


**Add code for Height, Weight, Co-Morbs, jobs, homebound, essential workers, adherence etc.**

In [ ]:
# del df['Unnamed: 0']
# del df['literacy']

# del df['residence'] 

# del df['geog'] 
# del df['working']

# del df['WorksAtSameCategory']

In [ ]:
df.rename(columns={'religion': 'Religion', 'caste': 'Caste'}, inplace=True)
df.rename(columns={'district': 'District', 'household_id': 'HHID'}, inplace=True)
df.rename(columns = {'WorkplaceID':'WorkPlaceID'}, inplace = True)

# df.rename(columns={'SchoolID': 'School_ID'}, inplace=True)

# df.rename(columns={'AdminUnitLatitude': 'AdminUnit_Lat', 'AdminUnitLongitude': 'AdminUnit_Lon'}, inplace=True)

# df.rename(columns={'PublicTransport_Jobs': 'Uses_PublicTransport',}, inplace=True)

# df.rename(columns={'essential_worker': 'Essential_Worker',}, inplace=True)

In [ ]:
df['WorkPlaceID'] = df['WorkPlaceID'].fillna(0)
df['School_ID'] = df['School_ID'].fillna(0)

In [ ]:
# replace it with np.round method
# df['Height'] = df['Height'].apply(lambda x:round(x,2))
# df['Weight'] = df['Weight'].apply(lambda x:round(x,2))

In [ ]:
#Uncomment the following in case these conversions are not done in previous steps in the pipeline


# df[['WorkPlaceID']] = df[['WorkPlaceID']].astype(pd.Int64Dtype())
# df[['School_ID']] = df[['School_ID']].astype(pd.Int64Dtype())
# df[['Agent_ID']] = df[['Agent_ID']].astype(pd.Int64Dtype())

In [ ]:
# df['StateLabel']='Haryana'
# df['Agent_ID'] = pd.Series(range(0, pop_size, 1)) + 6??00000000
# df['District'] = 'Ambala'

# df['HHID'] = df['HHID'] + 6??00000000   # First 3 digits is District Code: Pune- 998, Mumbai- 519, Ambala- 6??

In [ ]:
# If wokplaceid=0, and school_id != 0, then assign JobLabel: Student, JobID: 199

df['JobType'] = np.where((df['WorkPlaceID'] == 0) & (df['SchoolID'] != 0) , "Student", df['JobType'])
df.loc[df.JobType == "Student", 'JobID'] = 199

In [ ]:
# For Age=0, 1, 2, 3 JobType=Homebound

df['JobType'] = np.where((df['Age'] == 0), "Homebound", df['JobType'])
df['JobType'] = np.where((df['Age'] == 1), "Homebound", df['JobType'])
df['JobType'] = np.where((df['Age'] == 2), "Homebound", df['JobType'])
df['JobType'] = np.where((df['Age'] == 3), "Homebound", df['JobType'])

In [ ]:
# for homebound people JobID

df.loc[df.JobType == "Homebound", 'JobID'] = 0

In [ ]:
# for homebound people- School_id=0, school_lat=nan, school_long=nan, workplace_id=0, W_LAT, W_LON, essential worker=0

df.loc[df.JobType == "Homebound", ['essential_worker', 'WorkPlaceID', 'W_Lat', 'W_Lon', 'SchoolID', 'School_Lat', 'School_Lon' ]] = [0, 0, np.nan, np.nan, 0, np.nan, np.nan]

# Add Co-morbs

In [ ]:
# Add comorbs in the same proportion as in source- state in IHDS-II

# Need to get the IHDS filtered file for the state as source_df
'''

var = source_df.groupby(['M_Fever']).agg({'M_Fever': ['count']}).apply(lambda x: x/x.sum())
df['M_Fever'] = pd.Series(var['M_Fever'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Fever']).agg({'M_Fever': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Fever']).agg({'M_Fever': ['count']}).apply(lambda x: x/x.sum()))


var = source_df.groupby(['M_Cough']).agg({'M_Cough': ['count']}).apply(lambda x: x/x.sum())
df['M_Cough'] = pd.Series(var['M_Cough'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Cough']).agg({'M_Cough': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Cough']).agg({'M_Cough': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Diarrhea']).agg({'M_Diarrhea': ['count']}).apply(lambda x: x/x.sum())
df['M_Diarrhea'] = pd.Series(var['M_Diarrhea'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Diarrhea']).agg({'M_Diarrhea': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Diarrhea']).agg({'M_Diarrhea': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Cataract']).agg({'M_Cataract': ['count']}).apply(lambda x: x/x.sum())
df['M_Cataract'] = pd.Series(var['M_Cataract'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Cataract']).agg({'M_Cataract': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Cataract']).agg({'M_Cataract': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_TB']).agg({'M_TB': ['count']}).apply(lambda x: x/x.sum())
df['M_TB'] = pd.Series(var['M_TB'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_TB']).agg({'M_TB': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_TB']).agg({'M_TB': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_High_BP']).agg({'M_High_BP': ['count']}).apply(lambda x: x/x.sum())
df['M_High_BP'] = pd.Series(var['M_High_BP'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_High_BP']).agg({'M_High_BP': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_High_BP']).agg({'M_High_BP': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Heart_disease']).agg({'M_Heart_disease': ['count']}).apply(lambda x: x/x.sum())
df['M_Heart_disease'] = pd.Series(var['M_Heart_disease'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Heart_disease']).agg({'M_Heart_disease': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Heart_disease']).agg({'M_Heart_disease': ['count']}).apply(lambda x: x/x.sum()))


var = source_df.groupby(['M_Diabetes']).agg({'M_Diabetes': ['count']}).apply(lambda x: x/x.sum())
df['M_Diabetes'] = pd.Series(var['M_Diabetes'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Diabetes']).agg({'M_Diabetes': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Diabetes']).agg({'M_Diabetes': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Leprosy']).agg({'M_Leprosy': ['count']}).apply(lambda x: x/x.sum())
df['M_Leprosy'] = pd.Series(var['M_Leprosy'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Leprosy']).agg({'M_Leprosy': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Leprosy']).agg({'M_Leprosy': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Cancer']).agg({'M_Cancer': ['count']}).apply(lambda x: x/x.sum())
df['M_Cancer'] = pd.Series(var['M_Cancer'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Cancer']).agg({'M_Cancer': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Cancer']).agg({'M_Cancer': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Asthma']).agg({'M_Asthma': ['count']}).apply(lambda x: x/x.sum())
df['M_Asthma'] = pd.Series(var['M_Asthma'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Asthma']).agg({'M_Asthma': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Asthma']).agg({'M_Asthma': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Polio']).agg({'M_Polio': ['count']}).apply(lambda x: x/x.sum())
df['M_Polio'] = pd.Series(var['M_Polio'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Polio']).agg({'M_Polio': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Polio']).agg({'M_Polio': ['count']}).apply(lambda x: x/x.sum()))

var = source_df.groupby(['M_Paralysis']).agg({'M_Paralysis': ['count']}).apply(lambda x: x/x.sum())
df['M_Paralysis'] = pd.Series(var['M_Paralysis'].sample(n=pop_size,weights="count",replace=True).index)

print(source_df.groupby(['M_Paralysis']).agg({'M_Paralysis': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Paralysis']).agg({'M_Paralysis': ['count']}).apply(lambda x: x/x.sum()))


var = source_df.groupby(['M_Epilepsy']).agg({'M_Epilepsy': ['count']}).apply(lambda x: x/x.sum())
df['M_Epilepsy'] = pd.Series(var['M_Epilepsy'].sample(n=pop_size,weights="count",replace=True).index)
'''
print(source_df.groupby(['M_Epilepsy']).agg({'M_Epilepsy': ['count']}).apply(lambda x: x/x.sum()))
print(df.groupby(['M_Epilepsy']).agg({'M_Epilepsy': ['count']}).apply(lambda x: x/x.sum()))


In [ ]:
df['M_Epilepsy'] = df['M_Epilepsy'].replace(['No', 'Cured'], 0)
df['M_Epilepsy'] = df['M_Epilepsy'].replace(['Yes'], 1)

df['M_Fever'] = df['M_Fever'].replace(['No', 'Cured'], 0)
df['M_Fever'] = df['M_Fever'].replace(['Yes'], 1)

df['M_Cough'] = df['M_Cough'].replace(['No', 'Cured'], 0)
df['M_Cough'] = df['M_Cough'].replace(['Yes'], 1)

df['M_Diarrhea'] = df['M_Diarrhea'].replace(['No', 'Cured'], 0)
df['M_Diarrhea'] = df['M_Diarrhea'].replace(['Yes'], 1)

df['M_Cataract'] = df['M_Cataract'].replace(['No', 'Cured'], 0)
df['M_Cataract'] = df['M_Cataract'].replace(['Yes'], 1)

df['M_TB'] = df['M_TB'].replace(['No', 'Cured'], 0)
df['M_TB'] = df['M_TB'].replace(['Yes'], 1)

df['M_High_BP'] = df['M_High_BP'].replace(['No', 'Cured'], 0)
df['M_High_BP'] = df['M_High_BP'].replace(['Yes'], 1)

df['M_Heart_disease'] = df['M_Heart_disease'].replace(['No', 'Cured'], 0)
df['M_Heart_disease'] = df['M_Heart_disease'].replace(['Yes'], 1)

df['M_Diabetes'] = df['M_Diabetes'].replace(['No', 'Cured'], 0)
df['M_Diabetes'] = df['M_Diabetes'].replace(['Yes'], 1)

df['M_Leprosy'] = df['M_Leprosy'].replace(['No', 'Cured'], 0)
df['M_Leprosy'] = df['M_Leprosy'].replace(['Yes'], 1)

df['M_Cancer'] = df['M_Cancer'].replace(['No', 'Cured'], 0)
df['M_Cancer'] = df['M_Cancer'].replace(['Yes'], 1)

df['M_Asthma'] = df['M_Asthma'].replace(['No', 'Cured'], 0)
df['M_Asthma'] = df['M_Asthma'].replace(['Yes'], 1)

df['M_Polio'] = df['M_Polio'].replace(['No', 'Cured'], 0)
df['M_Polio'] = df['M_Polio'].replace(['Yes'], 1)

df['M_Paralysis'] = df['M_Paralysis'].replace(['No', 'Cured'], 0)
df['M_Paralysis'] = df['M_Paralysis'].replace(['Yes'], 1)

# Verify

This output goes as input to Mumbai Plot_Lat_Long_Shp_files.ipynb for further processing.

In [ ]:
df['WorkPlaceID'].isnull().values.any(),df['WorkPlaceID'].isnull().sum().sum(),df['WorkPlaceID'].isnull().T.any().T.sum()

In [ ]:
df[np.isnan(df['SchoolID'])]), df['WorkPlaceID'].isnull().T.any().T.sum()

In [ ]:
# df[df['Age'] == 0]['School_ID'].describe()
# df[df['Age'] < 3]['JobType'].describe()
# df[df['Age'] < 3]['JobType'].unique()
# df[df['JobType']=='Homebound']['WorkPlaceID']
# df[df['JobType']=='Homebound']['School_Lat'].describe()
# df[df['JobType']=='Homebound']['School_ID'].sum()
# df[df['JobType']=='Student'].describe()
var1 = 'Adherence_to_Intervention'
print((df[var1].value_counts()))
# print(df[var1][0:10])
print(df[var1].min())
df[var1].max()

# Geo-Plots

# Save the df into the specified format on Gdrive